# deal with the unbalanced dataset(negative:39684,positive:1443)

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
node_file = "/content/drive/My Drive/gnn_dp/final_project/data/raw/HIV_train.csv"

data = pd.read_csv(node_file )
data.index = data["index"]
data["HIV_active"].value_counts()
start_index = data.iloc[0]["index"]

# %% Apply oversampling

# Check how many additional samples we need
neg_class = data["HIV_active"].value_counts()[0]
pos_class = data["HIV_active"].value_counts()[1]
multiplier = int(neg_class/pos_class) - 1
print(neg_class)
print(pos_class)
print(multiplier)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from sklearn.utils import resample
positive_samples = data[data["HIV_active"] == 1]
negative_samples = data[data["HIV_active"] == 0]
positive_oversampled = pd.concat([positive_samples] * 2, ignore_index=True)
negative_downsampled = resample(negative_samples,
                                replace=False,
                                n_samples=len(positive_oversampled),
                                random_state=42)

balanced_data = pd.concat([positive_oversampled, negative_downsampled])
# Shuffle dataset
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
# Re-assign index (This is our ID later)
index = range(start_index, start_index + balanced_data.shape[0])
balanced_data.index = index
balanced_data["index"] = balanced_data.index
print(balanced_data["HIV_active"].value_counts(),len(balanced_data))

balanced_data.head()

In [ ]:
balanced_data.to_csv("/content/drive/My Drive/gnn_dp/final_project/data/raw/HIV_train_balanced.csv")

# create node and edge feature

In [ ]:
!pip install torch
!pip install torch_geometric
!pip install deepchem
!pip install dgl
!pip install pytorch-lightning
!pip install dm-haiku


In [1]:
#!pip install rdkit
import pandas as pd
from rdkit import Chem
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np
import os
from tqdm import tqdm

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")



class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data).
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            # Get node features
            node_feats = self._get_node_features(mol_obj)
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            # Get labels info
            label = self._get_labels(mol["HIV_active"])

            # Create data object
            data = Data(x=node_feats,
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol["smiles"]
                        )
            if self.test:
                torch.save(data,
                    os.path.join(self.processed_dir,
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data,
                    os.path.join(self.processed_dir,
                                 f'data_{index}.pt'))

    def _get_node_features(self, mol):
        """
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []

        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())

            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir,
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir,
                                 f'data_{idx}.pt'))
        return data

Torch version: 2.3.0+cu121
Cuda available: True
Torch geometric version: 2.6.1


In [ ]:
dataset = MoleculeDataset(root="/content/drive/My Drive/gnn_dp/final_project/data", filename="HIV_train_balanced.csv")

In [ ]:
data = dataset.get(1)
print(data)
print(data.edge_index.t())
print(data.x)
print(data.edge_attr)
print(data.y)

In [ ]:
test_dataset = MoleculeDataset(root="/content/drive/My Drive/gnn_dp/final_project/data", filename='HIV_test.csv', test=True)

# model building

## base model

In [2]:
#!pip install torch-geometric torch-sparse torch-scatter -q
#!pip install torch-geometric
import torch
from torch.nn import Module, Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool



In [3]:
def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for batch in tqdm(train_loader, desc=f"Epoch {epoch} Training"):
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model(batch)
        loss = loss_fn(pred, batch.y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        all_preds.append(pred.argmax(dim=1).cpu().numpy())
        all_labels.append(batch.y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="weighted")

    print(f"Epoch {epoch} Train Loss: {running_loss:.4f} | Accuracy: {accuracy:.4f} | F1 Score: {f1:.4f}")
    return running_loss, accuracy, f1

def test(model, test_loader, loss_fn):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            batch = batch.to(device)
            pred = model(batch)
            loss = loss_fn(pred, batch.y)
            running_loss += loss.item()

            all_preds.append(pred.argmax(dim=1).cpu().numpy())
            all_labels.append(batch.y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="weighted")

    print(f"Test Loss: {running_loss:.4f} | Accuracy: {accuracy:.4f} | F1 Score: {f1:.4f}")
    return running_loss, accuracy, f1


In [4]:
from torch.nn import BatchNorm1d
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool, global_max_pool

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

class GCNModel(Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pool_type="mean"):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

        self.norm1 = BatchNorm1d(hidden_dim)
        self.norm2 = BatchNorm1d(hidden_dim)

        self.fc = Linear(hidden_dim, output_dim)

        self.pool_type = pool_type

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.norm1(x)

        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.norm2(x)

        if self.pool_type == "mean":
            x = global_mean_pool(x, batch)
        elif self.pool_type == "sum":
            x = global_add_pool(x, batch)
        elif self.pool_type == "max":
            x = global_max_pool(x, batch)
        else:
            raise ValueError(f"Unsupported pooling type: {self.pool_type}")

        x = self.fc(x)
        return x



Running on device: cuda


In [5]:
def run_training_with_different_pools(run_id):
    train_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_train_balanced.csv")
    test_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename='HIV_test.csv', test=True)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    pool_types = ["mean", "sum", "max"]
    results = []

    for pool_type in pool_types:
        print(f"Training with pooling type: {pool_type}")

        model = GCNModel(
            input_dim=train_dataset[0].x.shape[1],
            hidden_dim=32,
            output_dim=2,
            pool_type=pool_type
        ).to(device)

        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        epochs = 5
        for epoch in range(1, epochs + 1):
            train_loss, train_acc, train_f1 = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
            test_loss, test_acc, test_f1 = test(model, test_loader, loss_fn)
            results.append({
                "pool_type": pool_type,
                "epoch": epoch,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "train_f1": train_f1,
                "test_loss": test_loss,
                "test_accuracy": test_acc,
                "test_f1": test_f1
            })

    filename = f"/vast/palmer/home.mccleary/md2675/mimic/final_project/data/pooling4_results_run_{run_id}.csv"
    df_results = pd.DataFrame(results)
    df_results.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

In [ ]:
for run_id in range(1,6): 
    run_training_with_different_pools(run_id)

Hierarchical Pooling method

In [12]:
from torch.nn import BatchNorm1d
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

class HierarchicalGCNModel(Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pool_ratio=0.5):
        super(HierarchicalGCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.pool1 = TopKPooling(hidden_dim, ratio=pool_ratio)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.pool2 = TopKPooling(hidden_dim, ratio=pool_ratio)

        self.norm1 = BatchNorm1d(hidden_dim)
        self.norm2 = BatchNorm1d(hidden_dim)

        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.norm1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)

        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.norm2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)

        x = global_add_pool(x, batch)
        x = self.fc(x)
        return x


Running on device: cuda


In [13]:
import pandas as pd

def run_hierarchical_pooling_training(run_id):
    # 加载数据
    train_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_train_balanced.csv")
    test_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename='HIV_test.csv', test=True)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = HierarchicalGCNModel(
        input_dim=train_dataset[0].x.shape[1],
        hidden_dim=32,
        output_dim=2,
        pool_ratio=0.5
    ).to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    results = []

    epochs = 5
    for epoch in range(1, epochs + 1):
        train_loss, train_acc, train_f1 = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
        test_loss, test_acc, test_f1 = test(model, test_loader, loss_fn)
        results.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "train_f1": train_f1,
            "test_loss": test_loss,
            "test_accuracy": test_acc,
            "test_f1": test_f1
        })

    filename = f"/vast/palmer/home.mccleary/md2675/mimic/final_project/data/hierarchical2_pooling_results_run_{run_id}.csv"
    df_results = pd.DataFrame(results)
    df_results.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

for run_id in range(1,6):  
    run_hierarchical_pooling_training(run_id)



Epoch 1 Training: 100%|██████████| 160/160 [00:04<00:00, 39.82it/s]


Epoch 1 Train Loss: 111.9719 | Accuracy: 0.5638 | F1 Score: 0.5635


Testing: 100%|██████████| 125/125 [00:02<00:00, 42.01it/s]


Test Loss: 72.0013 | Accuracy: 0.8642 | F1 Score: 0.8954


Epoch 2 Training: 100%|██████████| 160/160 [00:04<00:00, 37.67it/s]


...

Test Loss: 63.8236 | Accuracy: 0.8737 | F1 Score: 0.9023


Epoch 5 Training: 100%|██████████| 160/160 [00:04<00:00, 35.71it/s]


Epoch 5 Train Loss: 103.1859 | Accuracy: 0.6078 | F1 Score: 0.6019


Testing: 100%|██████████| 125/125 [00:02<00:00, 41.75it/s]

Test Loss: 71.1799 | Accuracy: 0.8055 | F1 Score: 0.8610
Results saved to /vast/palmer/home.mccleary/md2675/mimic/final_project/data/hierarchical2_pooling_results_run_5.csv


MPNN model

In [6]:
import torch
from torch.nn import Module, Linear, Sequential, ReLU, BatchNorm1d
from torch_geometric.nn import NNConv, TopKPooling, global_mean_pool
from torch_geometric.data import DataLoader
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ConfigurableHierarchicalGCNModel(Module):
    def __init__(self, input_dim, hidden_dim, output_dim, edge_dim, pool_ratio, dropout_rate):
        super(ConfigurableHierarchicalGCNModel, self).__init__()
        edge_network1 = Sequential(
            Linear(edge_dim, hidden_dim), 
            ReLU(),
            Linear(hidden_dim, input_dim * hidden_dim)
        )
        edge_network2 = Sequential(
            Linear(edge_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim * hidden_dim)
        )
        
        self.conv1 = NNConv(input_dim, hidden_dim, nn=edge_network1, aggr='mean')
        self.pool1 = TopKPooling(hidden_dim, ratio=pool_ratio)
        self.conv2 = NNConv(hidden_dim, hidden_dim, nn=edge_network2, aggr='mean')
        self.pool2 = TopKPooling(hidden_dim, ratio=pool_ratio)

        self.norm1 = BatchNorm1d(hidden_dim)
        self.norm2 = BatchNorm1d(hidden_dim)

        self.dropout = torch.nn.Dropout(dropout_rate)

        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        # print(f"x shape: {x.shape}")
        # print(f"edge_index shape: {edge_index.shape}") 
        # print(f"edge_attr shape: {edge_attr.shape}") 

        x = self.conv1(x, edge_index, edge_attr)  
        x = torch.relu(x)
        x = self.norm1(x)
        x = self.dropout(x)
        
        x, edge_index, edge_attr, batch, _, _ = self.pool1(x, edge_index, edge_attr, batch)
        # print(f"After pool1: x shape: {x.shape}, edge_index shape: {edge_index.shape}, edge_attr shape: {edge_attr.shape}")

        x = self.conv2(x, edge_index, edge_attr)
        x = torch.relu(x)
        x = self.norm2(x)
        x = self.dropout(x)

        x, edge_index, edge_attr, batch, _, _ = self.pool2(x, edge_index, edge_attr, batch)
        # print(f"After pool2: x shape: {x.shape}, edge_index shape: {edge_index.shape}, edge_attr shape: {edge_attr.shape}")

        x = global_sum_pool(x, batch)
        x = self.fc(x)
        return x



In [30]:
def generate_filename(run_id, params):
    param_str = "_".join([f"{key}-{params[key]}" for key in sorted(params.keys())])
    filename = f"/vast/palmer/home.mccleary/md2675/mimic/final_project/data/results2_run_{run_id}_{param_str}.csv"
    return filename

In [31]:
def train_and_evaluate_model(run_id, params):
    train_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_train_balanced.csv")
    test_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_test.csv", test=True)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = ConfigurableHierarchicalGCNModel(
        input_dim=train_dataset[0].x.shape[1],
        hidden_dim=params['model_embedding_size'],
        output_dim=2,
        edge_dim=train_dataset[0].edge_attr.shape[1],
        pool_ratio=params['pool_ratio'],
        dropout_rate=params["dropout_rate"]
    ).to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

    results = []

    epochs = params['epochs']
    for epoch in range(1, epochs + 1):
        train_loss, train_acc, train_f1 = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
        test_loss, test_acc, test_f1 = test(model, test_loader, loss_fn)
        results.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "train_f1": train_f1,
            "test_loss": test_loss,
            "test_accuracy": test_acc,
            "test_f1": test_f1
        })

    filename = generate_filename(run_id, params)
    df_results = pd.DataFrame(results)
    df_results.to_csv(filename, index=False)
    print(f"Results saved to {filename}")


In [32]:
import itertools
param_space = {
    "learning_rate": [0.01, 0.005],
    "model_embedding_size": [16, 32, 64],
    "pool_ratio": [0.4, 0.5, 0.6],
    "dropout_rate": [0, 0.2],
    "epochs": [5]
}
param_keys = list(param_space.keys())
param_values = list(param_space.values())
param_combinations = list(itertools.product(*param_values))

run_id = 1  # 全局运行编号
for param_id, param_values in enumerate(param_combinations, start=1):
    params = dict(zip(param_keys, param_values))
    print(f"Running configuration {param_id}: {params}")
    train_and_evaluate_model(run_id, params)
    run_id += 1

Running configuration 1: {'learning_rate': 0.01, 'model_embedding_size': 16, 'pool_ratio': 0.4, 'dropout_rate': 0, 'epochs': 5}


/home/md2675/.conda/envs/meditron/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Epoch 1 Training: 100%|██████████| 160/160 [00:04<00:00, 33.20it/s]


Epoch 1 Train Loss: 110.8304 | Accuracy: 0.5407 | F1 Score: 0.5399


Testing: 100%|██████████| 125/125 [00:03<00:00, 40.82it/s]


Test Loss: 77.2314 | Accuracy: 0.7672 | F1 Score: 0.8334


...

Test Loss: 83.6886 | Accuracy: 0.7109 | F1 Score: 0.7980


Epoch 5 Training: 100%|██████████| 160/160 [00:04<00:00, 37.84it/s]


Epoch 5 Train Loss: 104.0708 | Accuracy: 0.6174 | F1 Score: 0.6153


Testing: 100%|██████████| 125/125 [00:02<00:00, 43.45it/s]

Test Loss: 86.9650 | Accuracy: 0.6607 | F1 Score: 0.7617
Results saved to /vast/palmer/home.mccleary/md2675/mimic/final_project/data/results_run_36_dropout_rate-0.2_epochs-5_learning_rate-0.005_model_embedding_size-64_pool_ratio-0.6.csv


In [18]:
import itertools
import pandas as pd
import torch
from torch_geometric.data import DataLoader

def train_and_evaluate_model(run_id, params, repetitions=3):
    all_results = []

    for run in range(repetitions):
        print(f"Run {run + 1}/{repetitions} for params: {params}")

        train_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_train_balanced.csv")
        test_dataset = MoleculeDataset(root="/vast/palmer/home.mccleary/md2675/mimic/final_project/data", filename="HIV_test.csv", test=True)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        # 初始化模型、损失函数和优化器
        model = ConfigurableHierarchicalGCNModel(
            input_dim=train_dataset[0].x.shape[1],
            hidden_dim=params['model_embedding_size'],
            output_dim=2,
            edge_dim=train_dataset[0].edge_attr.shape[1],
            pool_ratio=params['pool_ratio'],
            dropout_rate=params["dropout_rate"]
        ).to(device)

        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

        results = []
        for epoch in range(1, params['epochs'] + 1):
            train_loss, train_acc, train_f1 = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
            test_loss, test_acc, test_f1 = test(model, test_loader, loss_fn)
            results.append({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "train_f1": train_f1,
                "test_loss": test_loss,
                "test_accuracy": test_acc,
                "test_f1": test_f1
            })
        all_results.append(pd.DataFrame(results))

    # 计算平均值和方差
    averaged_results = pd.concat(all_results).groupby("epoch").mean()
    variance_results = pd.concat(all_results).groupby("epoch").var()

    # 保存到文件
    avg_filename = f"/vast/palmer/home.mccleary/md2675/mimic/final_project/data/results_avg_run_{run_id}.csv"
    var_filename = f"/vast/palmer/home.mccleary/md2675/mimic/final_project/data/results_var_run_{run_id}.csv"
    averaged_results.to_csv(avg_filename, index=True)
    variance_results.to_csv(var_filename, index=True)
    print(f"Average results saved to {avg_filename}")
    print(f"Variance results saved to {var_filename}")

# 定义参数空间
param_space = {
    "learning_rate": [0.01, 0.005],
    "model_embedding_size": [16, 32, 64],
    "pool_ratio": [0.4, 0.5, 0.6],
    "dropout_rate": [0, 0.2],
    "epochs": [5]
}
param_keys = list(param_space.keys())
param_values = list(param_space.values())
param_combinations = list(itertools.product(*param_values))

# 逐个参数组合运行
run_id = 1
for param_id, param_values in enumerate(param_combinations, start=1):
    params = dict(zip(param_keys, param_values))
    print(f"Running configuration {param_id}: {params}")
    train_and_evaluate_model(run_id, params)
    run_id += 1


Running configuration 1: {'learning_rate': 0.01, 'model_embedding_size': 16, 'pool_ratio': 0.4, 'dropout_rate': 0, 'epochs': 5}
Run 1/3 for params: {'learning_rate': 0.01, 'model_embedding_size': 16, 'pool_ratio': 0.4, 'dropout_rate': 0, 'epochs': 5}


/home/md2675/.conda/envs/meditron/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Epoch 1 Training: 100%|██████████| 160/160 [00:04<00:00, 38.02it/s]


Epoch 1 Train Loss: 110.2340 | Accuracy: 0.5366 | F1 Score: 0.5358


Testing: 100%|██████████| 125/125 [00:02<00:00, 43.29it/s]


Test Loss: 90.0645 | Accuracy: 0.3906 | F1 Score: 0.5233


...

Testing: 100%|██████████| 125/125 [00:02<00:00, 43.32it/s]


Test Loss: 88.9686 | Accuracy: 0.4766 | F1 Score: 0.6083


Epoch 3 Training: 100%|██████████| 160/160 [00:04<00:00, 37.14it/s]


Epoch 3 Train Loss: 108.0675 | Accuracy: 0.5745 | F1 Score: 0.5745


Testing:  52%|█████▏    | 65/125 [00:01<00:01, 40.66it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

